Metacell run

In [ ]:
# Load the MetaCell data (host cell atlas)
load("E:/scTriH2/metacell/mc.scdr_TrH2_it4.Rda")

# Check what objects were loaded
ls()

# Assuming the main metacell object is called 'metacell_host', inspect it
str(metacell_host)

First, download the database from the github repository of xgrau:
- https://github.com/xgrau/placozoa-cell-type-evolution-code
- the directory is scatlas_results/data/scdb/
Then, load the metacell data as in read.me of the repo
- my data key is scdr_TrH2

In [ ]:
library(metacell)
setwd("/mnt/e/scTriH2/metacell")

# initialise metacell database using its relative path:
metacell::scdb_init("/mnt/e/scTriH2/metacell/mc_unzipped/", force_reinit = TRUE)

# read the mat object using its ID to refer to it; there ara analogous files for metacells, etc
mat_no_bact <- scdb_mat("scdr_TrH2_it2")

# sparse matrix available in the mat@mat slot, in this case it contains x genes and y cells
dim(mat_no_bact@mat)
    # 13948 genes x 15788 cells

# likewise, you may want to load a mc object containing cell-to-metacell assignments
mc = metacell::scdb_mc("scdr_TrH2_it4")
# the mc@mc slot is a vector with all cells and their associated metacell
length(mc@mc)
# 15704 cells associated with metacells

ctt = read.table("annotation_mc.TrH2.it4.reordered.tsv", header = TRUE)
head(ctt)


Secondly, the single cell data matrix (in this case in seurat object) is loaded. 
- The scRNA data are merged and processed (normalization , PCA)

In [ ]:
library(Seurat)
# load the symbiont sc matrix
summary_object_1 <- readRDS("../SEURAT/extracted/1.cellRangerCount/Seurat_outputs/merged_seurat_object.rds")
BiocManager::install("SingleCellExperiment")
library(SingleCellExperiment)
sce = as.SingleCellExperiment(summary_object_1, "sce")
bact_mat = scm_import_sce_to_mat(sce)

# Count bacterial signal per metacell
grellia <- colSums(as.matrix(bact_mat@mat, names(mc@mc)))
grellia_total <- tapply(grellia[names(mc@mc)], mc@mc, sum)

mc_counts=scr_mc_gene_counts(mc,no_bact_mat,5)
mc_sizes=colSums(mc_counts)
grellia_norm=grellia_total*100/(grellia_total+mc_sizes)



After counting:
Generate figures for the metacell bacterial signal


In [ ]:
# metacell bacterial signal
pdf("bacterial_signal_per_mc.pdf",h=8,w=8,useDingbats=F)
plot(grellia_norm, pch=20, cex=4, col=as.character(ctt$color))
text(grellia_norm, labels=names(grellia_norm), col=col=ifelse(pmax(grellia_norm)>4,"black",alpha("black",0)))
dev.off()

# sc bacterial signal 2D projections
grellia_mc2d <- scdb_mc2d(mc2d_id="scdr_TrH2_it4")
scp_plot_gene_2d_metacell_bacteria(mc2d,mc,grellia_norm,out_fn="Grellia_2d.png",plot_mc=F,log=F)

# fraction of infected/bacterial-containing cells per animal
cell_animal=as.vector(mat@cell_metadata[names(mc@mc),"dataset"])
names(cell_animal)=names(mc@mc)
frac_grellia_cells=table(cell_animal[grellia_positive_cells])*100/table(cell_animal)

pdf("Frac_infected_cells_per_animal.pdf",h=10,w=6,useDingbats=F)
par(mfrow=c(2,1))
barplot(frac_grellia_cells,ylab="% infected cells",col="gray30",ylim=c(0,10),main="Grellia")
dev.off()
